In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from obspy.signal.filter import lowpass

rng = np.random.default_rng(seed=12345)

def read_tgsfile(ist, dir):
    ist = str(ist).zfill(6)
    file_path = f"{dir}/tgs{ist}"
    eta = np.array([]) # do not add 0.0 as the value at t=0

    with open(file_path, 'r') as file:
        for line in file:
            parts = line.split()
            if "step=" in line:
                eta = np.append(eta, float(parts[4]))

    return eta

def lowpassfilter(eta, ts, ts_resample, resample):
    trend = (ts-ts[0])*(eta[-1]-eta[0])/(ts[-1]-ts[0]) + eta[0]
    eta_detrend = eta - trend
    f1 = 1/(ts[1]-ts[0])
    f2 = 1/(ts_resample[1]-ts_resample[0])
    lowpassed = lowpass(eta_detrend, f2/2, f1, zerophase=True) + trend
    return lowpassed[::resample]

def add_noise(eta, SN, rng):
    N = len(eta)
    eta_amp = np.abs(eta).max()
    sigma = eta_amp / SN
    noise = rng.normal(0.0, sigma, N)
    return eta + noise

In [ ]:
# config
SN = 10
fname_lonlatt_data = "lonlatt_data_20min.npy"
fname_eta_data = f"eta_data_20min_SN{SN}.npy"
fname_station_name = "../share/Snet133.txt"
fname_station_lonlat = "../share/Snet133_lonlat.txt"

In [ ]:
# station info
with open(fname_station_name) as f:
    Stations = [s.rstrip() for s in f.readlines()]
Nst = len(Stations)

lonlat_st = np.loadtxt(fname_station_lonlat)
lon_st = lonlat_st[:, 0]
lat_st = lonlat_st[:, 1]

In [2]:
# paramters of time series
dt = 0.1               # time step for FDM calculation [sec]
dt_resample = 10       # time interval for PINN [sec]
resample = int(dt_resample/dt) 

t1 = 20 * 60 # end of PINN waveform
t2 = 60 * 60 # end of FDM waveform 

Nt1 = int(t1/dt) + 1
Nt2 = int(t2/dt) + 1

ts1 = np.linspace(0, t1, Nt1)
ts1_resample = ts1[::resample]
ts2 = np.linspace(0, t2, Nt2)
ts2_resample = ts2[::resample]

Nt_data = len(ts1_resample)

In [ ]:
# read, filter, add noise and plot waveforms of S-net
os.makedirs(f"tgs_png_SN{SN}", exist_ok=True)
os.makedirs(f"tgs_filtered_SN{SN}", exist_ok=True)

scale = 10 

for ist in range(Nst):
    station = Stations[ist]

    eta = read_tgsfile(ist+1, "tgsfiles")
    eta_filtered = lowpassfilter(eta, ts2, ts2_resample, resample)
    eta_noised = add_noise(eta_filtered, SN, rng)
    
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(ts2_resample/60, eta_filtered, label="raw data")
    ax.plot(ts2_resample/60, eta_noised, label="noise added")
    ax.set_ylim(-scale,scale)
    ax.set_xlabel("Time [min]")
    ax.set_ylabel("Height [m]")
    ax.set_title(station)
    ax.legend()
    plt.savefig(f"tgs_png_SN{SN}/{station}.png")
    plt.close()

    np.savetxt(f"tgs_filtered_SN{SN}/{station}.txt", eta_noised)

In [12]:
# prepare PINN data
lon_data = np.array([])
lat_data = np.array([])
t_data = np.array([])
eta_data = np.array([])


for ist in range(Nst):
    lon_data = np.append(lon_data, np.full(Nt_data, lon_st[ist]))
    lat_data = np.append(lat_data, np.full(Nt_data, lat_st[ist]))
    t_data = np.append(t_data, ts1_resample)

    station = Stations[ist]
    eta = np.loadtxt(f"tgs_filtered_SN{SN}/{station}.txt")[:Nt_data]
    eta_data = np.append(eta_data, eta)


lonlatt_data = np.vstack((np.ravel(lon_data), np.ravel(lat_data), np.ravel(t_data))).T
np.save(fname_lonlatt_data, lonlatt_data)

eta_data = eta_data.reshape((Nst*Nt_data,1))
np.save(fname_eta_data, eta_data)

In [5]:
# read, filter and plot waveforms of coastal stations (used later)
fname_station_name = "../share/TohokuCoast14.txt"
with open(fname_station_name) as f:
    Stations_coast = [s.rstrip() for s in f.readlines()]

Nst_coast = len(Stations_coast)

os.makedirs(f"tgs_png_coast", exist_ok=True)
os.makedirs(f"tgs_filtered_coast", exist_ok=True)

scale = 15

for ist in range(Nst_coast):
    station = Stations_coast[ist]

    eta = read_tgsfile(ist+134, "tgsfiles")
    eta_filtered = lowpassfilter(eta, ts2, ts2_resample, resample)
    
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(ts2_resample/60, eta_filtered)
    ax.set_ylim(-scale,scale)
    ax.set_xlabel("Time [min]")
    ax.set_ylabel("Height [m]")
    ax.set_title(station)
    plt.savefig(f"tgs_png_coast/{station}.png")
    plt.close()

    np.savetxt(f"tgs_filtered_coast/{station}.txt", eta_filtered)